# kmeans.ipynb
Többdimenziós vektorok klaszterezéseit végzi el. A klaszterek száma 
folyamatosan nő, mindegyik klaszterszámra elvégzi a klaszterezést.  
A kimenet ugyanolyan, mint a bemenet, plusz az új oazlopok a klasztereket mutatják

In [1]:
#paraméterek
inputFile = 'el-diff0.tsv' # a bemeneti fájl .tsv, az első sor az oszlopok neve, az első oszlop a vektor azonosítója (lemma)
outputFile = 'el-clusters-diff' # a kimeneti fájlok nevei kiterjesztés nélkül. 
                            # Mindegyik (n) klaszterszámhoz létrehoz egy kimenetet *_n.tsv fájlnévvel
                            # ilyet mégsem csinál
                            # Létrehoz egy *_all.tsv fájlt is az összesített klaszterezéssel
outputDir = '' # ebbe a könyvtárba írja a kimeneteket
# outputDir = 'el-output/' # ebbe a könyvtárba írja a kimeneteket

clustersNumber = 20 # a klaszterek maximális száma (2 a legkevesebb)
clusterStep = 1 # a klaszterek számának lépésköze

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

from collections import Counter

In [3]:
# beolvassuk az input fájlt
data = pd.read_csv(inputFile, sep='\t')
data

,lemma,acc,dat,BAN,ON,RA,VAL,UL,BA,RÓL,...,NÁL,VÁ,IG,ÉRT,KÉNT,KOR,SZOR,NKÉNT,ADP,ADV
0,ad,-0.199580,-0.082895,0.018773,0.015383,-0.169512,-0.005893,-0.008201,-0.021562,-0.086502,...,-0.005679,0.000000,0.002095,0.061301,0.005922,-0.007653,0.000000,0.003715,0.032571,0.024985
1,alszik,-0.039451,0.003676,-0.059185,-0.033914,0.015771,-0.020738,-0.013507,-0.000960,0.000000,...,-0.000856,0.000000,-0.026275,0.002611,0.000000,0.001065,0.000000,-0.003091,0.048632,0.019245
2,bocsát,-0.118250,-0.026369,0.027129,-0.000884,-0.809102,0.030956,-0.051937,-0.062880,0.020627,...,0.004171,-0.002028,0.011442,-0.016227,0.008342,-0.008114,0.000000,-0.006085,0.072783,0.027929
3,bír,0.337465,-0.002367,-0.059379,-0.015592,-0.040237,-0.273092,0.024509,0.000000,-0.005917,...,-0.003550,0.000000,-0.030281,0.000000,-0.001183,-0.002367,0.000000,-0.001183,-0.041555,-0.127447
4,bírál,0.033032,0.005097,0.068126,0.032429,0.023616,0.036495,-0.247019,0.000000,-0.010738,...,-0.004027,0.000000,0.043478,-0.028188,0.011808,-0.006711,-0.001342,0.014493,-0.073320,-0.060189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,végez,0.299309,0.003691,-0.041533,-0.260688,0.007026,-0.071597,0.032206,-0.003222,-0.012421,...,-0.002890,0.000000,0.005015,0.002700,-0.008427,0.003691,0.000000,0.007685,-0.033225,0.021015
81,árul,-0.252037,0.014722,-0.185648,-0.198056,-0.001574,0.013889,-0.043426,-0.004630,0.195370,...,-0.003519,0.001481,-0.008611,-0.099074,-0.010093,0.001481,0.000000,-0.009259,-0.048889,-0.025185
82,ér,0.201323,-0.007711,0.092007,0.013280,-0.007333,-0.047729,-0.022019,-0.045614,-0.006184,...,0.014077,-0.002339,-0.001506,0.000238,-0.005134,-0.015086,0.000000,0.001646,0.023338,0.064638
83,ítél,0.033784,-0.340577,0.043332,0.005663,-0.534257,-0.003235,0.134479,0.001304,0.006432,...,-0.000817,0.000000,-0.006693,-0.010172,0.004311,-0.000817,0.000000,0.000000,-0.036304,0.104210


In [4]:
#kivesszük belőle a vektorokat
x = data.iloc[:,1:] # 1t for rows and second for columns
x

,acc,dat,BAN,ON,RA,VAL,UL,BA,RÓL,HOZ,...,NÁL,VÁ,IG,ÉRT,KÉNT,KOR,SZOR,NKÉNT,ADP,ADV
0,-0.199580,-0.082895,0.018773,0.015383,-0.169512,-0.005893,-0.008201,-0.021562,-0.086502,-0.034439,...,-0.005679,0.000000,0.002095,0.061301,0.005922,-0.007653,0.000000,0.003715,0.032571,0.024985
1,-0.039451,0.003676,-0.059185,-0.033914,0.015771,-0.020738,-0.013507,-0.000960,0.000000,-0.003091,...,-0.000856,0.000000,-0.026275,0.002611,0.000000,0.001065,0.000000,-0.003091,0.048632,0.019245
2,-0.118250,-0.026369,0.027129,-0.000884,-0.809102,0.030956,-0.051937,-0.062880,0.020627,-0.016227,...,0.004171,-0.002028,0.011442,-0.016227,0.008342,-0.008114,0.000000,-0.006085,0.072783,0.027929
3,0.337465,-0.002367,-0.059379,-0.015592,-0.040237,-0.273092,0.024509,0.000000,-0.005917,-0.002367,...,-0.003550,0.000000,-0.030281,0.000000,-0.001183,-0.002367,0.000000,-0.001183,-0.041555,-0.127447
4,0.033032,0.005097,0.068126,0.032429,0.023616,0.036495,-0.247019,0.000000,-0.010738,0.000000,...,-0.004027,0.000000,0.043478,-0.028188,0.011808,-0.006711,-0.001342,0.014493,-0.073320,-0.060189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.299309,0.003691,-0.041533,-0.260688,0.007026,-0.071597,0.032206,-0.003222,-0.012421,0.000386,...,-0.002890,0.000000,0.005015,0.002700,-0.008427,0.003691,0.000000,0.007685,-0.033225,0.021015
81,-0.252037,0.014722,-0.185648,-0.198056,-0.001574,0.013889,-0.043426,-0.004630,0.195370,0.002963,...,-0.003519,0.001481,-0.008611,-0.099074,-0.010093,0.001481,0.000000,-0.009259,-0.048889,-0.025185
82,0.201323,-0.007711,0.092007,0.013280,-0.007333,-0.047729,-0.022019,-0.045614,-0.006184,-0.014967,...,0.014077,-0.002339,-0.001506,0.000238,-0.005134,-0.015086,0.000000,0.001646,0.023338,0.064638
83,0.033784,-0.340577,0.043332,0.005663,-0.534257,-0.003235,0.134479,0.001304,0.006432,-0.001060,...,-0.000817,0.000000,-0.006693,-0.010172,0.004311,-0.000817,0.000000,0.000000,-0.036304,0.104210


In [9]:
dataCall = data.copy() # itt gyűjtjük az összesített klaszterezést

for knum in range(2, clustersNumber, clusterStep):
    kmeans = KMeans(knum, random_state=0)
    kmeans.fit(x)
    identified_clusters = kmeans.fit_predict(x)

#    dataC = data.copy()
#    dataC[str(knum)] = identified_clusters 
#    dataC.to_csv(outputDir + outputFile+"_"+str(knum)+".tsv", sep="\t", index=False)
    dataCall[str(knum)] = identified_clusters

dataCall.to_csv(outputDir + outputFile+'_all.tsv', sep="\t", index=False)